# Downloading & Preprocessing data


In [13]:
!bash ../data/download_data.sh 

--2025-04-26 22:18:53--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE186nnn/GSE186469/suppl/GSE186469_ctrl.data.txt.gz
Risoluzione di ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connessione a ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 11950751 (11M) [application/x-gzip]
Salvataggio in: «../data/raw/ctrl_data.txt.gz»

../data/raw/ctrl_da 100%[===================>]  11,40M  10,8MB/s    in 1,1s    

2025-04-26 22:18:54 (10,8 MB/s) - «../data/raw/ctrl_data.txt.gz» salvato [11950751/11950751]

--2025-04-26 22:18:54--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE186nnn/GSE186469/suppl/GSE186469_t2d.data.txt.gz
Risoluzione di ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.7, 130.14.250.10, ...
Connessione a ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:443... connesso.
Richiesta HTTP inviata, in attesa di ri